In [39]:
import os
import sys
import matplotlib.pyplot as plt
import numpy
import spatialpy

In [40]:
sys.path.append("../..")
%reload_ext autoreload
%autoreload 2

In [41]:
class Membrane(spatialpy.SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary
class GbgGradient(spatialpy.DataFunction):
    def __init__(self, Gbg_mid=5000, Gbg_slope=0.0, mem_vol=1.0):
        """ 1D domain from a to b. """
        spatialpy.DataFunction.__init__(self, name="GbgGradient")
        self.Gbg_mid = Gbg_mid
        self.Gbg_slope = Gbg_slope  # NM/um
        self.mem_vol = mem_vol
        
    def map(self, x):
        ret = self.Gbg_slope*x[2] + self.Gbg_mid
        return ret/self.mem_vol
    

In [42]:
def generatesphere(nx, ny):
        # Create mesh object
        obj = Mesh()
        # Vertices
        obj.vertices = numpy.zeros(( int(nx)*int(ny), 3), dtype=float)
        print(nx)
        print(ny)
        x_list = numpy.linspace(-1,1,nx)
        y_list = numpy.linspace(-1,1,ny)
        radius_external = 0.6
        radius_internal = 0.4
        ndx=0
        for x in x_list:
            for y in y_list:
                #r = numpy.sqrt(x*x + y*y)
                #if (r < radius_internal):
                    # make cytoplasm
                #elif (r < radius_external):
                    # make membrane
                #else:
                    # make water
                obj.vertices[ndx,0] = x        
                obj.vertices[ndx,1] = y
                obj.vertices[ndx,2] = 0.0
                ndx+=1
                
        # return model ref
        return obj
    
        # From https://bitbucket.org/bdrawert/ssa_sdpd-c-simulation-engine/src/master/examples/cdc42/Cdc42_S1a2.lmp
        #variable       Lx equal 10e-6                  #length x-dir
        #variable       Ly equal 10e-6                  #length y-dir

        #variable       radius_external equal 2.8e-6   #external radius
        #variable       radius_internal equal 2.2e-6   #internal radius

        #variable       nx equal 80                    #number of fluid particles in x-dir
        #variable       deltaf equal v_Ly/v_nx         #space between fluid particles
        #variable       deltac equal v_deltaf          #space between cell particles 

        #variable       densityf equal 1000            #density of fluid phase
        #variable       densityc equal 1000            #density of cell phase

In [43]:
class Cdc42_cycle_2D_SpatialPy_SDPD(spatialpy.Model):

    def __init__(self, model_name="Cdc42_2D-Moving_Mesh",Gbg_conc=5000.0, Gbg_pct_slope=0.0,delta1_val=0.00297,delta1_gbg_val=0.00297,delta_neg_val=1.0/30000, end_time=None, mesh=None, msh_filename=None,mesh_id=None,rotate=None):
        spatialpy.Model.__init__(self,model_name)

        D_membrane = 0.0053   #diffusion on the membrane from PLOS paper
        D_GDP = 0.0053
        D_GTP = 0.0053
        D_bulk = 10.0#50.0          #not sure of bulk diffusion const
        
        # Species
        Cdc24_m= spatialpy.Species(name="Cdc24_m",  diffusion_constant=D_membrane)
        Cdc24_c= spatialpy.Species(name="Cdc24_c",  diffusion_constant=D_bulk)
        Cdc42= spatialpy.Species(name="Cdc42",  diffusion_constant=D_GDP)
        Cdc42_a= spatialpy.Species(name="Cdc42_a",  diffusion_constant=D_GTP)
        Bem1_m = spatialpy.Species(name="Bem1_m",  diffusion_constant=D_membrane)
        Bem1_c = spatialpy.Species(name="Bem1_c",  diffusion_constant=D_bulk)
        Cla4 = spatialpy.Species(name="Cla4",  diffusion_constant=D_bulk)
        Cla4_a = spatialpy.Species(name="Cla4_a",  diffusion_constant=D_membrane)
        Cdc42_c = spatialpy.Species(name="Cdc42_c",  diffusion_constant=D_bulk)
        self.add_species([Cdc24_m, Cdc24_c, Cdc42, Cdc42_a, Bem1_m, Bem1_c, Cla4, Cla4_a, Cdc42_c])#,Gbg])
        
        cell_radius = 2.0

        self.mesh = generatesphere(80,80)

        # Parameters
        k_42d =   spatialpy.Parameter(name="k_42d", expression=1.0)#0.02)
        k_42a =   spatialpy.Parameter(name="k_42a", expression=0.2)#ORIGINAL: 0.2 #INCREASE: 0.2*100.0 #1e-05*membrane_volume)#*SA)
        k_24cm1 = spatialpy.Parameter(name="k_24cm1", expression=delta1_val)#ORIGINAL: delta1_val #3.3e-03*cytoplasm_volume)#*V)
        k_24mc =  spatialpy.Parameter(name="k_24mc", expression=0.35) #ORIGINAL: 0.35 #1.0)
        k_B1mc =  spatialpy.Parameter(name="k_B1mc", expression=0.35)#0.01)
        k_B1cm =  spatialpy.Parameter(name="k_B1cm", expression=0.2667)#ORIGINAL: 0.2667 #1e-05*cytoplasm_volume)#*V)
        k_Cla4a = spatialpy.Parameter(name="k_Cla4a", expression=0.006)
        k_Cla4d = spatialpy.Parameter(name="k_Cla4d", expression=0.01)
        k_24d =   spatialpy.Parameter(name="k_24d", expression=delta_neg_val)#1/3000#membrane_volume#SA
        
        beta1 = spatialpy.Parameter(name="beta1", expression=0.266) #ORIGINAL: 0.266  #INCREASE: 0.266*100.0
        beta2 = spatialpy.Parameter(name="beta2", expression=0.28)
        beta3 = spatialpy.Parameter(name="beta3", expression=1.0) #*10.0 #ORIGINAL: 1.0 #INCREASE: 1.0*10.0
        delta1_gbg = spatialpy.Parameter(name="delta1_gbg", expression=delta1_gbg_val)
        self.add_parameter([k_42a, k_42d, k_24cm1, k_24mc, k_B1mc, k_B1cm, k_Cla4a, k_Cla4d, k_24d,beta1, beta2, beta3,delta1_gbg])

        # Add Data Function to model the mating pheromone gradient.
        self.add_data_function(GbgGradient(Gbg_mid=Gbg_conc,Gbg_slope=Gbg_conc*Gbg_pct_slope/(cell_radius*2.0),mem_vol=membrane_volume))

        # Reactions
        #Gbg as species
        #Gbg as input function
        #DIFFERENCE WHETHER TO MULTIPLY BY 32 OR NOT
        CR0 = spatialpy.Reaction(name="CR0", reactants={Cdc24_c:1}, products={Cdc24_m:1}, propensity_function="delta1_gbg*Cdc24_c*GbgGradient*vol", restrict_to=2)
        CR1 = spatialpy.Reaction(name="CR1", reactants={Cdc24_c:1,Bem1_m:1}, products={Cdc24_m:1,Bem1_m:1},  rate=k_24cm1, restrict_to=2)
        CR2 = spatialpy.Reaction(name="CR2", reactants={Cdc24_m:1}, products={Cdc24_c:1}, rate=k_24mc, restrict_to=2)
        CR3 = spatialpy.Reaction(name="CR3", reactants={Cdc24_m:1,Cla4_a:1}, products={Cdc24_c:1,Cla4_a:1}, rate=k_24d, restrict_to=2)
        CR4 = spatialpy.Reaction(name="CR4", reactants={Cdc24_m:1,Cdc42:1}, products={Cdc24_m:1,Cdc42_a:1}, rate=k_42a, restrict_to=2)
        CR5 = spatialpy.Reaction(name="CR5", reactants={Cdc42_a:1}, products={Cdc42:1}, rate=k_42d, restrict_to=2)
        CR6 = spatialpy.Reaction(name="CR6", reactants={Cdc42_a:1,Bem1_c:1}, products={Cdc42_a:1,Bem1_m:1}, rate=k_B1cm, restrict_to=2)
        CR7 = spatialpy.Reaction(name="CR7", reactants={Bem1_m:1}, products={Bem1_c:1}, rate=k_B1mc, restrict_to=2)
        CR8 = spatialpy.Reaction(name="CR8", reactants={Cdc42_a:1,Cla4:1}, products={Cdc42_a:1,Cla4_a:1}, rate=k_Cla4a, restrict_to=2)
        CR9 = spatialpy.Reaction(name="CR9", reactants={Cla4_a:1}, products={Cla4:1}, rate=k_Cla4d, restrict_to=2)
        
        CR10 = spatialpy.Reaction(name="CR10", reactants={Cdc42_c:1}, products={Cdc42:1}, rate=beta2, restrict_to=2)
        CR11 = spatialpy.Reaction(name="CR11", reactants={Cdc42:1}, products={Cdc42_c:1}, rate=beta3, restrict_to=2)
        CR12 = spatialpy.Reaction(name="CR12", reactants={Cdc42_c:1,Cdc24_m:1}, products={Cdc42_a:1,Cdc24_m:1}, rate=beta1, restrict_to=2)
        self.add_reaction([CR0,CR1,CR2,CR3,CR4,CR5,CR6,CR7,CR8,CR9,CR10,CR11,CR12])
        
        self.restrict(species=Cdc24_c,subdomains=[1,2])
        self.restrict(species=Bem1_c,subdomains=[1,2])
        self.restrict(species=Cla4,subdomains=[1,2])
        self.restrict(species=Cdc24_m,subdomains=[2])
        self.restrict(species=Cdc42,subdomains=[2])
        self.restrict(species=Cdc42_a,subdomains=[2])
        self.restrict(species=Bem1_m,subdomains=[2])
        self.restrict(species=Cla4_a,subdomains=[2])
        self.restrict(species=Cdc42_c,subdomains=[1,2])
        
        # Scatter initial condition
        self.ScatterInitialCondition({Cdc42:2700},[2]) #10000  
        self.ScatterInitialCondition({Cdc24_c:1000},[1])    #We have 2000 Cdc24, GDI model has 1000
        self.ScatterInitialCondition({Bem1_c:3000},[1])
        self.ScatterInitialCondition({Cla4:5000},[1]) #5000
        self.ScatterInitialCondition({Cdc42_a:300},[2])
        
        if end_time is None:
            self.timespan(range(1000))
        else:
            if end_time >= 1.0:
                self.timespan(range(end_time))
            else:
                self.timespan(numpy.linspace(0,end_time,10))
            print("simulating to {0}s".format(end_time))